In [40]:

from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
import datasets
import torch
from datasets import Dataset, load_dataset, Sequence, ClassLabel, Features, Value
import evaluate
from preprocessing import preprocessing
from transformers import DataCollatorForTokenClassification
import numpy as np
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
df = preprocessing('en_ewt-up-dev.conllu')
label_list = list(df['label'].unique())

In [3]:
sent_df = df.groupby(['sentence_id']).agg(lambda x: x.tolist()).reset_index()

In [51]:
features = Features({
    'token_id': Sequence(feature=Value('float32')),
    'sentence_num': Sequence(feature=Value('int32')),
    'token': Sequence(feature=Value('string')),
    'lemma': Sequence(feature=Value('string')),
    'upos': Sequence(feature=Value('string')),
    'POS': Sequence(feature=Value('string')),
    'feats': Sequence(feature=Value('string')),
    'head': Sequence(feature=Value('string')),
    'deprel': Sequence(feature=Value('string')),
    'deps': Sequence(feature=Value('string')),
    'misc': Sequence(feature=Value('string')),
    'predicate': Sequence(feature=Value('string')),
    'predicate_token': Sequence(feature=Value('string')),
    'predicate_token_id': Sequence(feature=Value('int32')),
    'sentence_id': Value('int32'),
    'label': Sequence(feature=ClassLabel(names=label_list)),

})

ds = Dataset.from_pandas(sent_df[list(features.keys())], features=features)

/opt/homebrew/Caskroom/miniconda/base/envs/anlpa3/lib/python3.11/site-packages/datasets/load.py:753: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.17.1/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [52]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
SEP_TOKEN_ID = tokenizer.all_special_ids[tokenizer.all_special_tokens.index('[SEP]')]

In [53]:
from utils import tokenize_and_align_labels
tokenized_datasets = ds.map(lambda x: tokenize_and_align_labels(tokenizer, x))

Map: 100%|██████████| 4977/4977 [00:03<00:00, 1282.74 examples/s]


In [54]:
tokenized_datasets['labels'][0]

[-100, 0, 0, 1, 2, 0, 3, 0, -100]

In [55]:
from transformers import DataCollatorForTokenClassification

task = 'SRL'
batch_size = 32
model_name = 'bert-base-uncased'

args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
)

model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=len(label_list))

data_collator = DataCollatorForTokenClassification(tokenizer)


Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [56]:
import warnings
metric = evaluate.load("seqeval")


labels = [label_list[j] for l in ds["label"] for j in l]
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    m = metric.compute(predictions=[labels], references=[labels])
m

{'ADJ': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 243},
 'ADV': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 477},
 'ARG0': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 63},
 'ARG1': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 119},
 'ARG2': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 11},
 'ARG3': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 8},
 'ARGM-ADV': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'ARGM-CAU': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'ARGM-COM': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'ARGM-CXN': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 7},
 'ARGM-EXT': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'ARGM-LOC': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 13},
 'ARGM-MNR': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 3},
 'ARGM-TMP': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 8},
 'CAU': {

In [57]:
from utils import compute_metrics
td = tokenized_datasets.remove_columns(ds.column_names)
trainer = Trainer(
    model,
    args,
    train_dataset=td,
    eval_dataset=td,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


  0%|          | 0/468 [04:07<?, ?it/s]


In [58]:
trainer.train()

  1%|          | 5/468 [00:15<20:06,  2.61s/it]  